In [1]:
from pydicom import dcmread
import numpy as np
import os
import matplotlib.pyplot as plt
import re
import nibabel as nib
import SimpleITK as sitk
from helpers import *

In [2]:
class filtering:
    def __init__(self, segmentation): 
        self.segmentation=segmentation
        
    def get(self):
        array_1d = self.segmentation.flatten()
        unique_values, counts = np.unique(array_1d, return_counts=True)
        label_arrays = []
        for value in unique_values: 
            label_array = np.copy(self.segmentation) 
            label_array[np.where(label_array != value)] = 0 
            label_arrays.append(label_array)
        return label_arrays
            
seg_raw = sitk.ReadImage("D:/Documents/GitHub/VascuIAR/DeepLearning/data/MM_WHS/seg_res/1006/Segmentation.nii", sitk.sitkFloat32)
seg = sitk.GetArrayFromImage(seg_raw)
filtering_instance = filtering(seg)
label_array = filtering_instance.get()
explore_3D_array(label_array[2])
label_array[2].shape

interactive(children=(IntSlider(value=123, description='SLICE', max=247), Output()), _dom_classes=('widget-int…

(248, 512, 512)